In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for the dataset
train_dir = './dataset/train'
val_dir = './dataset/validation'

# Create ImageDataGenerators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize the images
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale for validation

# Load images from the directories and apply the transformations
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical'  # 'categorical' for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 3115 images belonging to 36 classes.
Found 351 images belonging to 36 classes.


In [30]:
# Load MobileNetV2 with pre-trained ImageNet weights, excluding the top layers
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to prevent its weights from being updated during the initial training
base_model.trainable = False

# Add custom layers on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')  # Output layer based on the number of classes
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [31]:
# Train the model with the training and validation data
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


d:\1A SEM 7\End\Computer Vision\Project\my_flask_app\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 7/97 ━━━━━━━━━━━━━━━━━━━━ 5:15 4s/step - accuracy: 0.1754 - loss: 3.5252

d:\1A SEM 7\End\Computer Vision\Project\my_flask_app\venv\Lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


97/97 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.6098 - loss: 1.3886 - val_accuracy: 0.8531 - val_loss: 0.4887
Epoch 2/10
 1/97 ━━━━━━━━━━━━━━━━━━━━ 1:34 982ms/step - accuracy: 0.6875 - loss: 1.1452

d:\1A SEM 7\End\Computer Vision\Project\my_flask_app\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


97/97 ━━━━━━━━━━━━━━━━━━━━ 19s 183ms/step - accuracy: 0.6875 - loss: 1.1452 - val_accuracy: 0.8562 - val_loss: 0.4760
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 238s 2s/step - accuracy: 0.7788 - loss: 0.7036 - val_accuracy: 0.8875 - val_loss: 0.3189
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 15s 148ms/step - accuracy: 0.7812 - loss: 0.8529 - val_accuracy: 0.8875 - val_loss: 0.3260
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 245s 3s/step - accuracy: 0.8271 - loss: 0.5227 - val_accuracy: 0.9281 - val_loss: 0.2529
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 17s 167ms/step - accuracy: 0.9375 - loss: 0.4806 - val_accuracy: 0.9125 - val_loss: 0.2820
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.8476 - loss: 0.4429 - val_accuracy: 0.9062 - val_loss: 0.2878
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 18s 177ms/step - accuracy: 0.7812 - loss: 0.6624 - val_accuracy: 0.9094 - val_loss: 0.2766
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.8790 - loss: 0.3558 - val_accuracy: 0.9125 - val_

In [32]:
# Unfreeze the last few layers of MobileNetV2 for fine-tuning
base_model.trainable = True

# Fine-tune only the top 10 layers of the base model
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Re-compile the model after unfreezing layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Use a smaller learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training the model
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/5
97/97 ━━━━━━━━━━━━━━━━━━━━ 252s 3s/step - accuracy: 0.8141 - loss: 0.5764 - val_accuracy: 0.9187 - val_loss: 0.2544
Epoch 2/5
97/97 ━━━━━━━━━━━━━━━━━━━━ 14s 136ms/step - accuracy: 0.9375 - loss: 0.3348 - val_accuracy: 0.9281 - val_loss: 0.2421
Epoch 3/5
97/97 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.8716 - loss: 0.4230 - val_accuracy: 0.9156 - val_loss: 0.2292
Epoch 4/5
97/97 ━━━━━━━━━━━━━━━━━━━━ 16s 158ms/step - accuracy: 0.8125 - loss: 0.4483 - val_accuracy: 0.9219 - val_loss: 0.2354
Epoch 5/5
97/97 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.8797 - loss: 0.3934 - val_accuracy: 0.9406 - val_loss: 0.1769


In [33]:
# Evaluate the model on the validation set
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation accuracy: {val_acc*100:.2f}%")


11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.9288 - loss: 0.2227
Validation accuracy: 92.88%


In [38]:
# Save the trained model
# In Colab
model.save('./custom_mobilenet_model.h5', save_format='tf')



In [35]:
from tensorflow.keras.models import load_model

In [39]:
import numpy as np
import tensorflow as tf

# Define custom class labels
class_labels = [
    'apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot',
    'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger',
    'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange',
    'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish',
    'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon'
]
model = load_model('custom_mobilenet_model.h5')


# Load the image for prediction
img_path = './images/Image_32.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

# Make predictions
predictions = model.predict(img_array)

# Sort the predictions and get the top 3
top_indices = predictions[0].argsort()[-3:][::-1]  # Get the indices of top 3 predictions
top_predictions = [(class_labels[i], predictions[0][i]) for i in top_indices]

# Print the top 3 predictions
for i, (label, score) in enumerate(top_predictions):
    print(f"{i+1}. {label}: {score:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 986ms/step
1. pear: 1.00
2. apple: 0.00
3. orange: 0.00
